In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "SF"
BARK_VAR = "WC"
FIA_SPCD = 11

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(0.0, 0.904973, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,5590.000000,5.590000e+03,5590.0,5590.000000,5590.000000,5590.000000,5590.000000,5590.000000,5590.000000,5590.000000,...,208.000000,208.000000,5590.000000,5590.000000,5590.000000,5590.000000,5590.0,5590.000000,5590.000000,5590.000000
mean,609.528623,4.989478e+11,11.0,105.888580,0.350549,3.245785,39.919141,12.550483,0.544628,0.516027,...,0.219952,0.465385,2005.494633,8.067263,2015.494633,7.679785,10.0,0.953309,5.147406,395.809660
std,10.112190,5.306046e+10,0.0,48.508184,0.234584,1.923897,9.832607,9.134721,0.220320,0.201699,...,0.195909,0.397339,2.398012,1.308731,2.398012,1.292090,0.0,0.616876,4.617650,240.254851
min,601.000000,4.100006e+11,11.0,4.400000,0.000000,0.000000,1.000000,5.000000,0.030000,0.010000,...,0.050000,0.050000,2001.000000,4.000000,2011.000000,4.000000,10.0,0.000000,0.000000,0.000000
25%,605.000000,4.103036e+11,11.0,67.500000,0.150000,1.483530,35.000000,6.400000,0.370000,0.350000,...,0.100000,0.200000,2004.000000,7.000000,2014.000000,7.000000,10.0,1.000000,1.000000,182.000000
50%,606.000000,5.305062e+11,11.0,102.520000,0.300000,3.316126,41.000000,8.900000,0.500000,0.500000,...,0.150000,0.300000,2005.000000,8.000000,2015.000000,8.000000,10.0,1.000000,4.000000,393.500000
75%,613.000000,5.307060e+11,11.0,142.500000,0.530000,4.974188,47.000000,14.500000,0.700000,0.660000,...,0.262500,0.600000,2008.000000,9.000000,2018.000000,9.000000,10.0,1.000000,10.000000,585.000000
max,800.000000,5.308078e+11,11.0,278.700000,1.500000,6.283185,65.000000,56.500000,0.990000,0.990000,...,1.050000,2.100000,2009.000000,12.000000,2019.000000,11.000000,10.0,4.000000,16.000000,824.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [10:40<00:00,  3.12it/s, 127 steps of size 2.92e-02. a


Done.
Samples saved at ../../models/maicf/SF//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/SF//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [30:02<00:00,  1.11it/s, 255 steps of size 1.53e-02. a


Done.
Samples saved at ../../models/maicf/SF//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/SF//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/SF//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/SF//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [14:22<00:00,  2.32it/s, 255 steps of size 2.72e-02. a


Done.
Samples saved at ../../models/maicf/SF//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/SF//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [30:27<00:00,  1.09it/s, 255 steps of size 1.72e-02. a


Done.
Samples saved at ../../models/maicf/SF//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/SF//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [11:13<00:00,  2.97it/s, 255 steps of size 2.56e-02. a


Done.
Samples saved at ../../models/maicf/SF//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/SF//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [09:34<00:00,  3.48it/s, 127 steps of size 2.76e-02. a


Done.
Samples saved at ../../models/maicf/SF//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/SF//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/SF//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/SF//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [20:28<00:00,  1.63it/s, 255 steps of size 1.81e-02. a


Done.
Samples saved at ../../models/maicf/SF//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/SF//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [19:50<00:00,  1.68it/s, 255 steps of size 1.62e-02. a


Done.
Samples saved at ../../models/maicf/SF//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/SF//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
